# Temporal Language Modeling

In [1]:
# File system
import os
import os.path
import shutil

import numpy as np
import time
from tqdm import tqdm
import regex

In [2]:
base_path = "data"
assert os.path.exists(base_path)

# SRILM

### Installation

In [3]:
assert os.path.exists('external')
SRILMpath = os.path.join(os.getcwd(), "external", "SRILM") # Does not work if directory contains spaces
#SRILMpath = "/Users/gnq144/OneDrive/Projekter/the_danish_connection/external/SRILM"

In [6]:
if not os.path.exists(SRILMpath):
    ! git clone https://github.com/gsayer/SRILM.git $SRILMpath
    ! gunzip -c $SRILMpath/srilm-1.7.1.tar.gz | tar xopf - -C $SRILMpath
    ! echo "SRILM = $SRILMpath" | cat - $SRILMpath/Makefile > temp && mv temp $SRILMpath/Makefile

#### Takes some time to build

In [5]:
%%bash -s "$SRILMpath"
cd $1
make World

Process is interrupted.


In [7]:
machine_type = !$SRILMpath/sbin/machine-type
machine_type = machine_type[0]
machine_type

'macosx'

In [8]:
# Give permission to execute
! chmod +x $SRILMpath/bin/$machine_type/*

In [9]:
!echo ""|sudo -S spctl --master-disable

Password:Sorry, try again.
Password:




In [10]:
! $SRILMpath/bin/$machine_type/ngram-count -help

Usage of command "/Users/gnq144/Desktop/the_danish_connection/external/SRILM/bin/macosx/ngram-count"
 -version:                 print version information
 -order:                   max ngram order
		Default value: 3
 -varprune:                pruning threshold for variable order ngrams
		Default value: 0
 -debug:                   debugging level for LM
		Default value: 0
 -recompute:               recompute lower-order counts by summation
 -sort:                    sort ngrams output
 -write-order:             output ngram counts order
		Default value: 0
 -tag:                     file tag to use in messages
 -text:                    text file to read
 -text-has-weights:        text file contains count weights
 -no-sos:                  don't insert start-of-sentence tokens
 -no-eos:                  don't insert end-of-sentence tokens
 -read:                    counts file to read
 -intersect:               intersect counts with this file
 -read-with-mincounts:     apply minimum cou

# Train language models

### Load datasets

In [11]:
def load_dataset(data_source_filename):
    print("Loading %s... " % data_source_filename.split("/")[-1], end="")
    t = time.time()
    dataset = dict()
    with np.load(data_source_filename, allow_pickle=True) as source_file:
        for key in source_file.keys():
          # print(key)
          dataset[key] = source_file[key].tolist()
        print("done (%.1fs)" % (time.time()-t), flush=True)
        return dataset

data_source_filenames = [os.path.join(base_path, fn) for fn in os.listdir(base_path)
                            if os.path.isfile(os.path.join(base_path, fn)) and fn[-3:]=='npz']
data_source_filenames

['data/SDHK_Swedish.npz',
 'data/StClare_dipl_danish.npz',
 'data/StClare_dipl_latin.npz',
 'data/Colonia.npz',
 'data/StClare_facs_latin.npz',
 'data/SemEval2015.npz',
 'data/SDHK_Latin.npz',
 'data/StClare_facs_danish.npz']

### Preprocessing functions

In [12]:
def prepare_for_wordlm(text):
    return text.lower()

def prepare_for_charlm(text, word_boundary="_"):
    text = regex.sub(" ", word_boundary, text) # Replace word boundary with special character
    return " ".join(list(text))

### Training functions

In [22]:
def initiate_lm_data(X, Y, preprocessor, train=[], val=[], test=False):
    if os.path.exists("tmp/"):
        shutil.rmtree("tmp/")

    lm_data_dir = "tmp/model_data/"
    if not os.path.exists(lm_data_dir): os.makedirs(lm_data_dir)
    
    
    if test:
        doc_dir = "tmp/documents/"   
        if not os.path.exists(doc_dir): os.makedirs(doc_dir)
    
        # Write all docs to files
        for doc, i in zip(X, range(len(X))):
            with open(doc_dir+str(i)+".txt", "w") as w:
                w.write(preprocessor(doc))
        return
                

    # Create train and dev folds
    for label in np.unique(Y):
        os.mkdir(lm_data_dir+str(label)+"/")
        open(lm_data_dir+str(label)+"/train.txt", "w").close()
        open(lm_data_dir+str(label)+"/val.txt", "w").close()    

    for x, y, setname in zip(np.append(X[train], X[val]), np.append(Y[train], Y[val]), np.append(["train"]*len(train), ["val"]*len(val))):
        destfile = lm_data_dir+str(y)+"/"+setname+".txt"
        with open(destfile, "a") as w:
            w.write(preprocessor(x)+"\n")

def train_temporal_lm(data_folder, dest_folder, model_name, srilm, order):
    
    for category_folder, _, _ in os.walk(data_folder):
        if category_folder == data_folder: # Skip root
            continue
        
        category = category_folder.split("/")[-1]
        dest_model = dest_folder+model_name+"_"+category+".lm"
        print("Training model @", dest_model, "(using data {})".format(category_folder+"/trainval.txt"))        
        ! cat $category_folder/train.txt $category_folder/val.txt > $category_folder/trainval.txt
        
        error_msg = !$srilm/ngram-count -order $order -lm $dest_model -text $category_folder/trainval.txt -kndiscount -interpolate # -unk -map-unk "<UNK>"
        if error_msg:
            !$srilm/ngram-count -order $order -lm $dest_model -text $category_folder/trainval.txt -interpolate
            print("Error while discounting ({})".format(";".join(error_msg)))
            
    

### Experimental setup

In [23]:
from itertools import product

In [24]:
orders = (1, 2, 3, 5)
levels = ("word", "char")
bin_widths = (25,)

configs = []
for order, level, bin_width in product(orders, levels, bin_widths):

    configs.append({
        # lm
        "level" : level,
        "preprocessor" : prepare_for_charlm if level == "char" else prepare_for_wordlm,
        "order" : order,
        "srilm" : SRILMpath+"/bin/"+machine_type+"/",

        # classification
        "bin_width" : bin_width,

        # data
        "model_dir" : "models/",
        "model_name" : "lm_SRILM_{name}_{level}_n{order}_{bin_width}",
    })

### Train

In [25]:
for data_source_filename in data_source_filenames:
    dataset = load_dataset(data_source_filename)
    dataname = data_source_filename[data_source_filename.rfind("/")+1:data_source_filename.rfind(".")]
    
    try:
        X = np.array(list(map(lambda x: x["text"], dataset['data'])))
    except KeyError:
        X = np.array(list(map(lambda x: "\n".join([" ".join(sentence) for sentence in x["tokens"]]), dataset['data'])))    
  
    
    for config in configs:
        print(config)
        print()
        
        if not os.path.exists(config["model_dir"]): os.makedirs(config["model_dir"])
        
        Y = np.asarray([np.mean(item['date']) if type(item['date']) is tuple else item['date'] for item in dataset['data']], np.int)
        Y = config["bin_width"]*(Y//config["bin_width"])

        print("Classes in data: %s" % np.unique(Y))
        print()

        # Initiate training date
        print("Initiating text data.. " , end="")
        t = time.time()
        initiate_lm_data(X, Y, config["preprocessor"], dataset["folds"]["train"], dataset["folds"]["val"])
        print("done (%.1fs)" % (time.time()-t), flush=True)

        # Train models    
        train_temporal_lm(data_folder="tmp/model_data/",
                          dest_folder=config["model_dir"],
                          model_name=config["model_name"].format(name=dataname,
                                                                   level=config["level"],
                                                                   order=config["order"],
                                                                   bin_width=config["bin_width"]),
                          srilm=config["srilm"],
                          order=config["order"])
        print()
        print()
    

Loading SDHK_Swedish.npz... done (0.8s)
{'level': 'word', 'preprocessor': <function prepare_for_wordlm at 0x7f9e5f0e3b90>, 'order': 1, 'srilm': '/Users/gnq144/Desktop/the_danish_connection/external/SRILM/bin/macosx/', 'bin_width': 25, 'model_dir': 'models/', 'model_name': 'lm_SRILM_{name}_{level}_n{order}_{bin_width}'}

Classes in data: [1150 1200 1225 1250 1275 1300 1325 1350 1375 1400 1425 1450 1475 1500]

Initiating text data.. done (1.0s)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n1_25_1450.lm (using data tmp/model_data/1450/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n1_25_1200.lm (using data tmp/model_data/1200/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n1_25_1375.lm (using data tmp/model_data/1375/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n1_25_1500.lm (using data tmp/model_data/1500/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n1_25_1350.lm (using data tmp/model_data/1350/trainval.txt

Training model @ models/lm_SRILM_SDHK_Swedish_word_n2_25_1150.lm (using data tmp/model_data/1150/trainval.txt)
Error while discounting (one of modified KneserNey discounts is negative;error in discount estimator for order 1)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n2_25_1300.lm (using data tmp/model_data/1300/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n2_25_1275.lm (using data tmp/model_data/1275/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n2_25_1425.lm (using data tmp/model_data/1425/trainval.txt)


{'level': 'char', 'preprocessor': <function prepare_for_charlm at 0x7f9e5f0eb320>, 'order': 2, 'srilm': '/Users/gnq144/Desktop/the_danish_connection/external/SRILM/bin/macosx/', 'bin_width': 25, 'model_dir': 'models/', 'model_name': 'lm_SRILM_{name}_{level}_n{order}_{bin_width}'}

Classes in data: [1150 1200 1225 1250 1275 1300 1325 1350 1375 1400 1425 1450 1475 1500]

Initiating text data.. done (1.4s)
Training model @ models/lm_SRILM_

Training model @ models/lm_SRILM_SDHK_Swedish_word_n3_25_1500.lm (using data tmp/model_data/1500/trainval.txt)
Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 2)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n3_25_1350.lm (using data tmp/model_data/1350/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n3_25_1225.lm (using data tmp/model_data/1225/trainval.txt)
Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 3)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n3_25_1475.lm (using data tmp/model_data/1475/trainval.txt)
Error while discounting (one of modified KneserNey discounts is negative;error in discount estimator for order 1)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n3_25_1400.lm (using data tmp/model_data/1400/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n3_25_1250.lm (using data tmp/

Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 1)
Training model @ models/lm_SRILM_SDHK_Swedish_char_n3_25_1400.lm (using data tmp/model_data/1400/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Swedish_char_n3_25_1250.lm (using data tmp/model_data/1250/trainval.txt)
Error while discounting (one of modified KneserNey discounts is negative;error in discount estimator for order 1)
Training model @ models/lm_SRILM_SDHK_Swedish_char_n3_25_1325.lm (using data tmp/model_data/1325/trainval.txt)
Error while discounting (one of modified KneserNey discounts is negative;error in discount estimator for order 1)
Training model @ models/lm_SRILM_SDHK_Swedish_char_n3_25_1150.lm (using data tmp/model_data/1150/trainval.txt)
Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 1)
Training model @ models/lm_SRILM_SDHK_Swedish_char_n3_25_1300.lm (using data t

Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 2)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n5_25_1350.lm (using data tmp/model_data/1350/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n5_25_1225.lm (using data tmp/model_data/1225/trainval.txt)
GT discounting disabled
GT discounting disabled
Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 3)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n5_25_1475.lm (using data tmp/model_data/1475/trainval.txt)
GT discounting disabled
GT discounting disabled
Error while discounting (one of modified KneserNey discounts is negative;error in discount estimator for order 1)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n5_25_1400.lm (using data tmp/model_data/1400/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Swedish_word_n5_25_1250.lm (using data tmp/model_data/1250

...

Error while discounting (one of modified KneserNey discounts is negative;error in discount estimator for order 1)
Training model @ models/lm_SRILM_SDHK_Latin_char_n3_25_1425.lm (using data tmp/model_data/1425/trainval.txt)
Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 1)


{'level': 'word', 'preprocessor': <function prepare_for_wordlm at 0x7f9e5f0e3b90>, 'order': 5, 'srilm': '/Users/gnq144/Desktop/the_danish_connection/external/SRILM/bin/macosx/', 'bin_width': 25, 'model_dir': 'models/', 'model_name': 'lm_SRILM_{name}_{level}_n{order}_{bin_width}'}

Classes in data: [1100 1125 1150 1175 1200 1225 1250 1275 1300 1325 1350 1375 1400 1425
 1450 1475 1500]

Initiating text data.. done (3.0s)
Training model @ models/lm_SRILM_SDHK_Latin_word_n5_25_1450.lm (using data tmp/model_data/1450/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Latin_word_n5_25_1200.lm (using data tmp/model_data/1200/trainval.txt)
Training 

Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 1)
Training model @ models/lm_SRILM_SDHK_Latin_char_n5_25_1200.lm (using data tmp/model_data/1200/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Latin_char_n5_25_1375.lm (using data tmp/model_data/1375/trainval.txt)
Training model @ models/lm_SRILM_SDHK_Latin_char_n5_25_1125.lm (using data tmp/model_data/1125/trainval.txt)
Error while discounting (one of modified KneserNey discounts is negative;error in discount estimator for order 1)
Training model @ models/lm_SRILM_SDHK_Latin_char_n5_25_1500.lm (using data tmp/model_data/1500/trainval.txt)
Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 1)
Training model @ models/lm_SRILM_SDHK_Latin_char_n5_25_1100.lm (using data tmp/model_data/1100/trainval.txt)
Error while discounting (one of modified KneserNey discounts is negative;error in discount 

Error while discounting (one of modified KneserNey discounts is negative;error in discount estimator for order 1)
Training model @ models/lm_SRILM_StClare_facs_danish_char_n1_25_1550.lm (using data tmp/model_data/1550/trainval.txt)
Training model @ models/lm_SRILM_StClare_facs_danish_char_n1_25_1425.lm (using data tmp/model_data/1425/trainval.txt)
Error while discounting (one of modified KneserNey discounts is negative;error in discount estimator for order 1)


{'level': 'word', 'preprocessor': <function prepare_for_wordlm at 0x7f9e5f0e3b90>, 'order': 2, 'srilm': '/Users/gnq144/Desktop/the_danish_connection/external/SRILM/bin/macosx/', 'bin_width': 25, 'model_dir': 'models/', 'model_name': 'lm_SRILM_{name}_{level}_n{order}_{bin_width}'}

Classes in data: [1400 1425 1450 1475 1500 1525 1550 1575]

Initiating text data.. done (0.0s)
Training model @ models/lm_SRILM_StClare_facs_danish_word_n2_25_1450.lm (using data tmp/model_data/1450/trainval.txt)
Training model @ models/lm_SRILM_StClar

Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 4)
Training model @ models/lm_SRILM_StClare_facs_danish_word_n5_25_1525.lm (using data tmp/model_data/1525/trainval.txt)
Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 4)
Training model @ models/lm_SRILM_StClare_facs_danish_word_n5_25_1500.lm (using data tmp/model_data/1500/trainval.txt)
Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 3)
Training model @ models/lm_SRILM_StClare_facs_danish_word_n5_25_1475.lm (using data tmp/model_data/1475/trainval.txt)
Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 4)
Training model @ models/lm_SRILM_StClare_facs_danish_word_n5_25_1400.lm (using data tmp/model_data/1400/trainval.txt)
Error while discounting (one

Error while discounting (one of required modified KneserNey count-of-counts is zero;error in discount estimator for order 3)


{'level': 'char', 'preprocessor': <function prepare_for_charlm at 0x7f9e5f0eb320>, 'order': 5, 'srilm': '/Users/gnq144/Desktop/the_danish_connection/external/SRILM/bin/macosx/', 'bin_width': 25, 'model_dir': 'models/', 'model_name': 'lm_SRILM_{name}_{level}_n{order}_{bin_width}'}

Classes in data: [1400 1425 1450 1475 1500 1525 1550 1575]

Initiating text data.. done (0.1s)
Training model @ models/lm_SRILM_StClare_facs_danish_char_n5_25_1450.lm (using data tmp/model_data/1450/trainval.txt)
Training model @ models/lm_SRILM_StClare_facs_danish_char_n5_25_1525.lm (using data tmp/model_data/1525/trainval.txt)
Error while discounting (one of modified KneserNey discounts is negative;error in discount estimator for order 1)
Training model @ models/lm_SRILM_StClare_facs_danish_char_n5_25_1500.lm (using data tmp/model_data/1500/trainval.txt)
Training model @ models/lm_S

## Testing

In [26]:
import pandas as pd
from sklearn.metrics import accuracy_score

results = dict()

In [27]:
for data_source_filename in set(data_source_filenames).difference(set(results.keys())):
#for data_source_filename in data_source_filenames:
    dataset = load_dataset(data_source_filename)
    #dataset.pop("temporallm", None) # Remove old results
    
    dataname = data_source_filename[data_source_filename.rfind("/")+1:data_source_filename.rfind(".")]
    
    try:
        X = np.array(list(map(lambda x: x["text"], dataset['data'])))
    except KeyError:
        X = np.array(list(map(lambda x: "\n".join([" ".join(sentence) for sentence in x["tokens"]]), dataset['data'])))    

    results[data_source_filename] = dict()
    for config in tqdm(configs):
        
        Y = np.asarray([np.mean(item['date']) if type(item['date']) is tuple else item['date'] for item in dataset['data']], np.int)        
        
        print(config)
        print()
        Y = config["bin_width"]*(Y//config["bin_width"])        
        DM = pd.DataFrame(index=dataset["folds"]["test"], columns=np.unique(Y), dtype=float)
        
        initiate_lm_data(X, Y, config["preprocessor"], test=True)

        for label in np.unique(Y):
            print("Evaluating model {}.. ".format(label))        
            t = time.time()
            model_name = config["model_name"].format(name=dataname,
                                                level=config["level"],
                                                order=config["order"],
                                                bin_width=config["bin_width"])+"_"+str(label)
            model=config["model_dir"]+model_name+".lm"
            for test_i in dataset["folds"]["test"]:
                test_file = "tmp/documents/"+str(test_i)+".txt"

                order = config["order"]
                srilm = config["srilm"]
                try:
                    _, out = !$srilm/ngram -order $order -lm $model -ppl $test_file

                    

                    ppl = float(regex.search("(?<=ppl= +)\d+(\.\d+)?", out).group(0))

                except ValueError:
                    out = !$srilm/ngram -order $order -lm $model -ppl $test_file
                    print("Out (ValueError)", out)
                    ppl=np.inf

                DM.loc[test_i, label] = ppl
        print(DM)
        predictions = DM.idxmin(axis=1)
        results[data_source_filename]["{level}_{order}_{width}".format(level=config["level"],
                                                 order=config["order"],
                                                 width=config["bin_width"])] = {
            "y_pred" : predictions,
            "y_true" : Y[dataset["folds"]["test"]],        
            "accuracy": accuracy_score(Y[dataset["folds"]["test"]], predictions)
        }
        print(results[data_source_filename]["{level}_{order}_{width}".format(level=config["level"],
                                                 order=config["order"],
                                                 width=config["bin_width"])]["accuracy"])
        

Loading StClare_dipl_latin.npz... done (0.1s)


  0%|          | 0/8 [00:00<?, ?it/s]

{'level': 'word', 'preprocessor': <function prepare_for_wordlm at 0x7f9e5f0e3b90>, 'order': 1, 'srilm': '/Users/gnq144/Desktop/the_danish_connection/external/SRILM/bin/macosx/', 'bin_width': 25, 'model_dir': 'models/', 'model_name': 'lm_SRILM_{name}_{level}_n{order}_{bin_width}'}

Evaluating model 1250.. 
Evaluating model 1275.. 
Evaluating model 1300.. 
Evaluating model 1325.. 
Evaluating model 1350.. 
Evaluating model 1375.. 
Evaluating model 1400.. 
Evaluating model 1425.. 
Evaluating model 1450.. 
Evaluating model 1475.. 
Evaluating model 1500.. 
Evaluating model 1525.. 


 12%|█▎        | 1/8 [00:13<01:35, 13.69s/it]

         1250     1275     1300     1325     1350     1375     1400     1425  \
189   586.253  624.116  653.907  478.970  211.881  413.864  493.706  183.932   
162   470.858  295.404  394.893  446.262  231.453  343.961  383.841  156.701   
139   803.206  452.107  604.362  559.295  307.452  403.563  516.241  209.534   
216   378.436  388.306  341.067  197.127  154.891  350.995  304.866  191.483   
91    967.859  644.092  793.744  518.934  244.476  392.787  428.933  219.730   
..        ...      ...      ...      ...      ...      ...      ...      ...   
252   526.911  436.856  437.882  525.521  288.096  469.079  555.581  212.165   
5     863.884  543.864  703.210  784.804  455.327  696.207  905.136  252.875   
4    1234.670  784.420  816.692  611.086  305.463  668.330  566.236  291.887   
23    883.738  460.331  657.059  656.805  406.313  520.009  553.864  259.994   
352  1123.770  808.927  961.629  824.205  440.628  615.801  700.988  332.589   

        1450     1475      1500     152

...

        1400      1425      1450      1475      1500      1525      1550  \
15  11.30660  10.02000   8.92439   7.94183   8.10411   8.66032  10.88520   
86   9.51041  10.22070  10.55450   8.95863   9.73141  12.29930  21.71060   
10   9.19864   8.64630   8.54090   8.88540   9.38180  10.94890  17.91540   
6    8.24461   6.51955   7.08686   7.39987   7.79604   8.37377  15.14850   
68   8.83894   8.25180   7.90821   9.12099   9.11158  11.04840  15.19530   
79   8.48610   9.00304   8.78552   8.59918   8.36653  13.25360  26.10870   
90   8.45769   7.57230   6.60888   7.84052   8.23172  10.56670  18.67190   
7    9.79381   8.44913   7.59366   7.02241   7.37270   9.19698  15.50020   
30   9.64723   8.73031   7.46469   8.12690   7.61360  10.72350  16.67630   
28   9.72811   8.75809   8.55826   8.30689   7.40349   9.41406  14.24070   
82   9.67355   8.99246   9.28830   8.71685   8.45760  12.34680  20.84310   
92   8.92609   8.03766   7.27490   7.38877   7.49077   9.44300  15.52610   
74  11.84900

 88%|████████▊ | 7/8 [00:21<00:02,  2.99s/it]

       1400     1425     1450     1475     1500     1525      1550      1575
15  279.565  157.851  136.156  190.487  185.053  143.431  243.3100  247.8520
86  383.763  177.598  267.319  166.866  173.387  135.997  150.8030  154.7190
10  184.916  217.181  430.038  219.815  295.952  270.514  168.5620  237.3640
6   275.853  137.837  224.378  187.361  178.189  195.643  211.6050  198.1230
68  144.413  162.301  250.818  332.014  171.487  276.002  264.8170  175.7060
79  288.408  256.580  284.807  207.494  221.866  375.666  174.0700  211.7790
90  203.513  154.096  150.834  169.234  197.942  197.923  200.5270  170.5880
7   289.422  142.820  156.009  200.377  183.226  267.031  164.9210  152.1140
30  187.659  156.266  245.701  192.299  241.035  262.334  227.6270  203.2630
28  200.755  108.468  173.849  155.529  158.538  191.947  244.0240  190.7250
82  162.697  131.951  222.012  166.551  119.486  293.200  458.1110  195.1890
92  170.545  153.096  172.141  177.770  183.994  174.632  261.5120  150.9420

100%|██████████| 8/8 [00:24<00:00,  3.08s/it]

        1400      1425      1450      1475      1500      1525      1550  \
15  10.48760   9.72504   8.26594   7.32687   6.78920   7.93200  10.70830   
86   7.58990   9.10763   9.73471   7.35775   7.98720  11.31760  22.20530   
10   8.21226   7.41539   7.64254   8.49396   8.73993  10.25150  18.55310   
6    7.15745   5.70120   5.93901   6.40367   6.58938   7.60844  15.82820   
68   7.71225   7.41749   6.64649   8.12212   7.98299  10.90960  15.32660   
79   6.68526   7.43871   7.73169   7.42444   6.75735  12.19690  26.48220   
90   7.16899   6.63806   4.89397   6.76127   7.07242   9.95042  19.70090   
7    8.45346   7.49125   6.25519   5.66798   6.07007   8.11468  15.60580   
30   8.79399   8.54946   6.45924   7.16933   6.18829   9.79050  17.26890   
28   8.25614   8.13304   7.48962   7.17430   5.95068   8.66232  15.40970   
82   7.35325   7.81293   7.34686   6.45471   6.25996  11.18280  21.98330   
92   7.80484   7.61650   6.13319   6.36527   6.29121   8.63568  16.55660   
74  10.60040

In [28]:
#    print("Saving results..")
#    np.savez_compressed(data_source_filename, **dataset, temporallm=results)
#    print()
#    print()
for data_source_filename in results.keys():
    #try:
    dataset = load_dataset(data_source_filename)
    print(" Saving %s... " % data_source_filename.split("/")[-1], end="")
    t = time.time()
    if 'temporallm' in dataset.keys():
        dataset.pop('temporallm')
    np.savez_compressed(data_source_filename, **dataset, temporallm=results[data_source_filename])
    print("done (%.1fs)" % (time.time()-t), flush=True)
    #except:
    #    pass

Loading StClare_dipl_latin.npz... done (0.2s)
 Saving StClare_dipl_latin.npz... done (0.3s)
Loading StClare_facs_latin.npz... done (0.1s)
 Saving StClare_facs_latin.npz... done (0.5s)
Loading SDHK_Swedish.npz... done (0.9s)
 Saving SDHK_Swedish.npz... done (3.3s)
Loading SemEval2015.npz... done (0.5s)
 Saving SemEval2015.npz... done (1.8s)
Loading Colonia.npz... done (8.2s)
 Saving Colonia.npz... done (18.6s)
Loading StClare_facs_danish.npz... done (0.1s)
 Saving StClare_facs_danish.npz... done (0.2s)
Loading SDHK_Latin.npz... done (2.1s)
 Saving SDHK_Latin.npz... done (7.5s)
Loading StClare_dipl_danish.npz... done (0.0s)
 Saving StClare_dipl_danish.npz... done (0.1s)


In [29]:
print("Done")

Done
